In [ ]:
#@Author: Gemma Gordon
#@Date: 2022
# Script to analyse structural clustering by RMSD and loop length 
# Structural clustering code from https://github.com/brennanaba/cdr-structural-clustering

Structural clustering output gives, for all of H1 H2 and H3 for both Abs and Nbs, the PDB structure and their cluster (a representative PDB structure). This representative structure will be matched with a canonical form. We want to know how structurally different Abs and Nbs loops are/whether they occupy different structural space. Some clusters contain both Abs and Nbs structures. To find if this is significant, we can randomly sample the clusters and find the number/proportion of structures that fall into these 'overlap' clusters by chance. We will use this as a baseline with which to compare the actual number of structures that fall into overlap clusters. 

In [1]:
import pandas as pd
import numpy as np

Load in structural clustering results as dataframe

In [2]:
results = pd.read_csv("CDR_loop_clusters.csv").drop('Unnamed: 0', axis=1)

Separate into H1, H2 and H3 results

In [3]:
results_h1 = results.loc[results['H_loop'] == 'H1']
results_h2 = results.loc[results['H_loop'] == 'H2']
results_h3 = results.loc[results['H_loop'] == 'H3']

In [4]:
results_h1.columns

Index(['PDB ID', 'cluster_by_RMSD', 'Ig_ID', 'cluster_by_length', 'H_loop'], dtype='object')

In [5]:
results_h1.to_csv('results_h1')
results_h2.to_csv('results_h2')
results_h3.to_csv('results_h3')

For the results for each loop, need to:

- identify which clusters have Nbs and Abs members in them
- find how many overlap clusters
- find how many & what proportion of structures belong to overlap clusters

In [6]:
def get_freqs(results_df):

    # get breakdown of abs vs nbs within clusters
    freq_df = results_df.groupby(['cluster_by_RMSD', 'Ig_ID']).size().reset_index()
    freq_df = freq_df.pivot(index='cluster_by_RMSD', columns='Ig_ID')[0].fillna(0).sort_values(by='Abs', ascending=False)

    return freq_df

In [7]:
def find_overlap_clusters(results_df):

    # input results for a loop and get frequencies of Abs and Nbs
    freq_df = get_freqs(results_df)
    # identify overlap clusters 
    overlaps_id = []
    overlaps_abs, overlaps_nbs = 0, 0 

    for row in freq_df.itertuples():
        # if there are members from Abs and Nbs in the cluster
        # row[1] is Abs and row[2] is Nbs
        if row[1] != 0 and row[2] != 0:
            # row[0] is PDB ID
            overlaps_id.append(row[0])
            # how many structures in overlapping cluster?
            overlaps_abs += int(row[1])
            overlaps_nbs += int(row[2])
    
    # get proportion of total structures that are in overlap clusters
    # NOTE want proportion (1) just abs (2) just nbs (3) abs and nbs 
    overlaps_props = dict()
    overlaps_props['Abs'] = overlaps_abs / len(results_df)
    overlaps_props['Nbs'] = overlaps_nbs / len(results_df)
    overlaps_props['Both'] = (overlaps_abs + overlaps_nbs) / len(results_df)

    return overlaps_id, overlaps_props

In [8]:
overlaps_id_h1, overlaps_prop_h1 = find_overlap_clusters(results_h1)
overlaps_id_h2, overlaps_prop_h2 = find_overlap_clusters(results_h2)
overlaps_id_h3, overlaps_prop_h3 = find_overlap_clusters(results_h3)

In [9]:
print('H1 PROPORTION OF STRUCTURES IN OVERLAP:', overlaps_prop_h1)
print('H2 PROPORTION OF STRUCTURES IN OVERLAP:', overlaps_prop_h2)
print('H3 PROPORTION OF STRUCTURES IN OVERLAP:', overlaps_prop_h3)

H1 PROPORTION OF STRUCTURES IN OVERLAP: {'Abs': 0.58311345646438, 'Nbs': 0.1345646437994723, 'Both': 0.7176781002638523}
H2 PROPORTION OF STRUCTURES IN OVERLAP: {'Abs': 0.6543535620052771, 'Nbs': 0.21108179419525067, 'Both': 0.8654353562005277}
H3 PROPORTION OF STRUCTURES IN OVERLAP: {'Abs': 0.051011433597185574, 'Nbs': 0.010554089709762533, 'Both': 0.06156552330694811}


In [10]:
all_h1_freqs, all_h2_freqs, all_h3_freqs = get_freqs(results_h1), get_freqs(results_h2), get_freqs(results_h3)

In [11]:
for i, loop in enumerate([all_h1_freqs, all_h2_freqs, all_h3_freqs]):

    print(i)

    #how many Abs and Nbs clusters overlap, how many clusters contain only Abs or Nbs?
    print('Total:', len(loop))
    print('Only Abs:', len(loop.loc[(loop["Abs"] != 0) & (loop["Nbs"] == 0)]))
    print('Only Nbs:', len(loop.loc[(loop["Abs"] == 0) & (loop["Nbs"] != 0)]))
    print('Overlap:', len(loop.loc[(loop["Abs"] != 0) & (loop["Nbs"] != 0)]))
    #how many clusters are singletons?
    print('Abs singletons:', len(loop.loc[(loop["Abs"] == 1) & (loop["Nbs"] == 0)]))
    print('Nbs singletons:', len(loop.loc[(loop["Nbs"] == 1) & (loop["Abs"] == 0)]))

0
Total: 168
Only Abs: 66
Only Nbs: 84
Overlap: 18
Abs singletons: 48
Nbs singletons: 64
1
Total: 94
Only Abs: 35
Only Nbs: 36
Overlap: 23
Abs singletons: 21
Nbs singletons: 25
2
Total: 729
Only Abs: 489
Only Nbs: 230
Overlap: 10
Abs singletons: 383
Nbs singletons: 197


Generate sets of random clusters that match sizes of original clusters and see how many by chance overlap in Abs and Nbs members

In [57]:
import pandas as pd
from tqdm import tqdm

def get_cluster_sizes(cluster_file):
    '''Returns array with sizes of multiple occupancy clusters'''
    df = pd.read_csv(cluster_file, index_col=0)
    cluster_size = df.groupby('cluster_by_RMSD').nunique()['PDB ID'].sort_values(ascending=False)
    return cluster_size[cluster_size > 1].values

def sample_clusters_from_db(db, cluster_sizes):
    new_db = pd.DataFrame()

    for i, cluster_size in enumerate(cluster_sizes):
        cluster = db.sample(n=cluster_size, replace=False)
        cluster['cluster'] = str(i)
        new_db = pd.concat([new_db, cluster])
        db = db.drop(cluster.index)
    return new_db

def generate_random_clusters(cluster_size, n_replicates, db):
    '''Returns list of dataframes with Abs randomly assigned to clusters'''
    random_clusters = []

    db['cluster'] = 'None'

    for i in tqdm(range(n_replicates)):
        random_clusters.append(sample_clusters_from_db(db, cluster_size))

    return random_clusters

creates list of cluster sizes 

In [58]:
cluster_sizes_h1 = get_cluster_sizes('struc_clustering/results_h1')
cluster_sizes_h2 = get_cluster_sizes('struc_clustering/results_h2')
cluster_sizes_h3 = get_cluster_sizes('struc_clustering/results_h3')

In [59]:
cluster_sizes_h1

array([498, 201,  46,  21,  18,  16,  15,  15,  12,  11,  11,  11,  11,
         8,   8,   7,   7,   6,   6,   5,   5,   5,   4,   4,   4,   4,
         3,   3,   3,   3,   3,   3,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2])

generates 20 sets of random clusters of same sizes as original cluster sizes, returns list of dfs

In [60]:
random_clusters_h1 = generate_random_clusters(cluster_sizes_h1, 20, results_h1)
random_clusters_h2 = generate_random_clusters(cluster_sizes_h2, 20, results_h2)
random_clusters_h3 = generate_random_clusters(cluster_sizes_h3, 20, results_h3)

<ipython-input-57-586cc62a7013>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['cluster'] = 'None'
100%|██████████| 20/20 [00:03<00:00,  5.22it/s]


In [61]:
len(random_clusters_h1)

20

For each DF in results, find how many overlap and which overlap to find proportions

for each df, get list of overlap clusters

In [62]:
def get_overlap_ids_list(random_clusters):

    overlap_ids = []
    for rc in random_clusters:
        overlap_id_df, _ = find_overlap_clusters(rc)
        overlap_ids.append(overlap_id_df)

    return overlap_ids

In [63]:
random_overlap_ids_h1 = get_overlap_ids_list(random_clusters_h1)
random_overlap_ids_h2 = get_overlap_ids_list(random_clusters_h2)
random_overlap_ids_h3 = get_overlap_ids_list(random_clusters_h3)

In [64]:
len(random_overlap_ids_h1)

20

Label PDB structures in results as whether overlap or non-overlap cluster

In [65]:
def add_overlap_label(results_df, overlap_ids):
    
    output_df = results_df.copy(deep=True)
    overlap_labels = []
    for row in output_df.itertuples():
        # if cluster_by_RMSD column (representative structure for cluster) is in the overlap list:
        if row[2] in overlap_ids:
            overlap_labels.append('Overlap')
        else:
            overlap_labels.append('Non-overlap')
            
    # create new column in results df
    output_df['Overlap_label'] = overlap_labels

    return output_df

In [66]:
def find_overlaps(random_clusters, overlap_ids):

    overlap_dfs = []
    for rc, ids in zip(random_clusters, overlap_ids):
        overlap_df = add_overlap_label(rc, ids)
        overlap_dfs.append(overlap_df)

    return overlap_dfs

In [67]:
random_clusters_h1_l = find_overlaps(random_clusters_h1, random_overlap_ids_h1)
random_clusters_h2_l = find_overlaps(random_clusters_h2, random_overlap_ids_h2)
random_clusters_h3_l = find_overlaps(random_clusters_h3, random_overlap_ids_h3)

In [68]:
len(random_clusters_h1_l)

20

Count how many clusters are overlap clusters in the random clusters & find the mean and stdev

for each labelled df, get freq df

In [69]:
random_clusters_h1_l[0].loc[(random_clusters_h1_l[0]['Overlap_label'] == 'Overlap')]

,PDB ID,cluster_by_RMSD,Ig_ID,cluster_by_length,H_loop,cluster,Overlap_label
286,5wi9,5do2,Abs,13_0_0_,H1,0,Overlap
508,6vbg,5do2,Nbs,13_0_0_,H1,0,Overlap
713,5en2,1ahw,Abs,13_0_0_,H1,0,Overlap
622,5bo1,1ahw,Abs,13_0_0_,H1,0,Overlap
432,5y2l,5do2,Abs,13_0_0_,H1,0,Overlap
...,...,...,...,...,...,...,...
158,6pxh,5do2,Abs,13_0_0_,H1,53,Overlap
330,6j5f,5do2,Abs,13_0_0_,H1,53,Overlap
370,4lsu,5do2,Abs,13_0_0_,H1,54,Overlap
737,1fsk,1ahw,Abs,13_0_0_,H1,54,Overlap


find total number of clusters which are randomly overlap clusters

In [70]:
def get_overlap_summary(dfs):

    overlap_counts = []
    for df in dfs:
        #how many Abs and Nbs clusters overlap?
        overlap_df = df[['cluster_by_RMSD', 'Overlap_label']].drop_duplicates(subset='cluster_by_RMSD', keep='first')
        overlap_count = len(overlap_df.loc[(overlap_df['Overlap_label'] == 'Overlap')])
        overlap_counts.append(overlap_count)

    # get mean and stdev overlap
    mean_overlap = np.mean(overlap_counts)
    stdev_overlap = np.std(overlap_counts)

    summary = {'MEAN': mean_overlap, 'STDEV': stdev_overlap}

    return summary

In [71]:
h1_summary = get_overlap_summary(random_clusters_h1_l)
h2_summary = get_overlap_summary(random_clusters_h2_l)
h3_summary = get_overlap_summary(random_clusters_h3_l)

In [72]:
print('H1 RANDOM OVERLAP RESULTS:', h1_summary)
print('H2 RANDOM OVERLAP RESULTS:', h2_summary)
print('H3 RANDOM OVERLAP RESULTS:', h3_summary)

H1 RANDOM OVERLAP RESULTS: {'MEAN': 16.2, 'STDEV': 1.2884098726725126}
H2 RANDOM OVERLAP RESULTS: {'MEAN': 22.25, 'STDEV': 0.82915619758885}
H3 RANDOM OVERLAP RESULTS: {'MEAN': 3.3, 'STDEV': 1.5524174696260025}
